In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd 'gdrive/MyDrive/SER/implementations/pepino21_interspeech/'

/content/gdrive/MyDrive/SER/implementations/pepino21_interspeech


In [3]:
!pip uninstall torch torchaudio torchvision torchtext -y

Found existing installation: torch 2.2.1+cu121
Uninstalling torch-2.2.1+cu121:
  Successfully uninstalled torch-2.2.1+cu121
Found existing installation: torchaudio 2.2.1+cu121
Uninstalling torchaudio-2.2.1+cu121:
  Successfully uninstalled torchaudio-2.2.1+cu121
Found existing installation: torchvision 0.17.1+cu121
Uninstalling torchvision-0.17.1+cu121:
  Successfully uninstalled torchvision-0.17.1+cu121
Found existing installation: torchtext 0.17.1
Uninstalling torchtext-0.17.1:
  Successfully uninstalled torchtext-0.17.1


In [4]:
!pip install torch==1.11.0 torchaudio==0.11.0 torchvision==0.12.0 torchtext==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 94.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.1 requires torch>=2, but you have torch 1.11.0 which is incompatible.


In [5]:
!pip install opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168840 sha256=0bfca68f4f12df2fe98ae444db88d4d9a12198be614317184a8bb65b926d91bd
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639


In [6]:
import torch
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn.utils.rnn import pad_sequence

from dataset_classes.ravdess import MergeCalmAndNeutralRAVDESS, MyResample, RavdessAudio
from models.dense.pt_all2 import PreTrainedAllLayers

In [7]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

device

device(type='cuda')

In [8]:
def my_collate_fn(batch):
    sample_list = [sample[0] for sample in batch]
    batch_tensor = pad_sequence(sample_list, batch_first=True)
    lengths = torch.tensor([sample[1] for sample in batch], dtype=torch.int)
    samp_rates = torch.tensor([sample[2] for sample in batch], dtype=torch.int)
    spkr_ids = torch.tensor([sample[3] for sample in batch], dtype=torch.int)
    labels = torch.tensor([sample[4] for sample in batch], dtype=torch.long)
    return batch_tensor, lengths, samp_rates, spkr_ids, labels

In [9]:
def train_epoch(dataloader, model, loss_fn, optimizer, print_every, lr_scheduler):
    """
    print_every: after how many batches to print loss (on the last processed batch)
    """
    num_samples = len(dataloader.dataset)
    samples_done = 0
    for batch_idx, batch in enumerate(dataloader):
        model_inp = tuple(batch[i].to(device) for i in range(4))
        labels = batch[4].to(device)

        preds = model(model_inp)
        loss = loss_fn(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        samples_done += model_inp[0].size(0)
        if (batch_idx+1)%print_every == 0:
            print(f"loss: {loss:.5f}    [{samples_done:4d}/{num_samples:4d}]")
    lr_scheduler.step()

In [10]:
def test_epoch(dataloader, num_classes, model, loss_fn):
    num_batches = len(dataloader)
    num_samples = len(dataloader.dataset)

    loss = 0
    acc = 0
    tp = torch.zeros((num_classes))
    gnd_trth_p = torch.zeros((num_classes))

    model.eval()
    for batch in dataloader:
        model_inp = tuple(batch[i].to(device) for i in range(4))
        labels = batch[4].to(device)

        with torch.no_grad():
            preds = model(model_inp)
            loss += loss_fn(preds, labels).item()
            preds = preds.argmax(dim=1)
            acc += (preds == labels).sum().item()
            for i in range(num_classes):
                tp[i] += torch.logical_and(preds==i, labels==i).sum().item()
                gnd_trth_p[i] += (labels == i).sum().item()
    model.train()

    loss /= num_batches
    acc /= num_samples
    recall = tp/gnd_trth_p
    avg_recall = torch.mean(recall)
    print("Test Error:")
    print(f"    loss: {loss:.5f}, acc: {100*acc:.2f} %\n    avg recall: {100*avg_recall:.2f} %")
    return loss

In [11]:
train_dataset = RavdessAudio(dir_path="../../datasets/",
                             csv_path="../../datasets/RAVDESS/my_stuff/train_csv.csv",
                             transform=MyResample(48000, 16000),
                             target_transform=MergeCalmAndNeutralRAVDESS())

val_dataset = RavdessAudio(dir_path="../../datasets/",
                           csv_path="../../datasets/RAVDESS/my_stuff/val_csv.csv",
                           transform=MyResample(48000, 16000),
                           target_transform=MergeCalmAndNeutralRAVDESS())

test_dataset = RavdessAudio(dir_path="../../datasets/",
                            csv_path="../../datasets/RAVDESS/my_stuff/test_csv.csv",
                            transform=MyResample(48000, 16000),
                            target_transform=MergeCalmAndNeutralRAVDESS())

num_classes = 7

In [12]:
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=my_collate_fn,
                                               shuffle=True,
                                               generator=torch.Generator(device))

val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=my_collate_fn,
                                             shuffle=True,
                                             generator=torch.Generator(device))

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              collate_fn=my_collate_fn,
                                              shuffle=True,
                                              generator=torch.Generator(device))

In [13]:
# load mean, std_dev
norm_dict = torch.load('./normalization_tensors/ravdess/pt_w2v2/all_layers_spkr.pt')

w2v2_mean = norm_dict['mean']
w2v2_std_dev = norm_dict['std_dev']

for k in w2v2_mean.keys():
    w2v2_mean[k] = w2v2_mean[k].to(device)
for k in w2v2_std_dev.keys():
    w2v2_std_dev[k] = w2v2_std_dev[k].to(device)

In [14]:
#        dim:      16,      32,      64,     128
#   val_loss: xxxxxxx, xxxxxxx, xxxxxxx, xxxxxxx
#    val_acc:   xxxxx,   xxxxx,   xxxxx,   xxxxx
# val_recall:   xxxxx,   xxxxx,   xxxxx,   xxxxx
# dim1 = 16

model = PreTrainedAllLayers(
    250,
    # dim1,
    norm_means=w2v2_mean,
    norm_std_devs=w2v2_std_dev,
)
model.to(device);

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

In [15]:
loss_fn = nn.CrossEntropyLoss()

In [16]:
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = ExponentialLR(optimizer, gamma=0.95)

In [17]:
num_epochs = 30
patience = 4

print("Before Training, on val set:")
_ = test_epoch(val_dataloader, num_classes, model, loss_fn)
print("Before Training, on test set:")
_ = test_epoch(test_dataloader, num_classes, model, loss_fn)

print("Starting training")
best_loss = float('inf')
i = 0
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    print("--------------------------------")
    train_epoch(train_dataloader, model, loss_fn, optimizer, 4, lr_scheduler)
    loss = test_epoch(val_dataloader, num_classes, model, loss_fn)
    if loss < best_loss:
        best_loss = loss
        torch.save({
            "epochs_done": epoch+1,
            "model_state_dict": model.state_dict(),
            "opt_state_dict": optimizer.state_dict(),
            "val_loss": loss
            }, "./saved_models/dense/best_pt_all_spkr.pt")
        i = 0
    elif loss > best_loss:
        i += 1
    if i > patience:
        print(f"Early stopping after {epoch+1} epochs")
        break;

Before Training, on val set:
Test Error:
    loss: 1.95935, acc: 7.69 %
    avg recall: 14.29 %
Before Training, on test set:
Test Error:
    loss: 1.95928, acc: 7.69 %
    avg recall: 14.29 %
Starting training
Epoch 1
--------------------------------
loss: 1.91924    [ 256/2036]
loss: 1.90991    [ 512/2036]
loss: 1.83795    [ 768/2036]
loss: 1.73277    [1024/2036]
loss: 1.60289    [1280/2036]
loss: 1.49459    [1536/2036]
loss: 1.54843    [1792/2036]
loss: 1.25618    [2036/2036]
Test Error:
    loss: 1.42441, acc: 50.48 %
    avg recall: 39.88 %
Epoch 2
--------------------------------
loss: 1.37063    [ 256/2036]
loss: 1.34267    [ 512/2036]
loss: 1.22270    [ 768/2036]
loss: 1.10349    [1024/2036]
loss: 1.20382    [1280/2036]
loss: 1.10143    [1536/2036]
loss: 1.08567    [1792/2036]
loss: 0.90992    [2036/2036]
Test Error:
    loss: 0.94959, acc: 66.35 %
    avg recall: 64.29 %
Epoch 3
--------------------------------
loss: 0.84648    [ 256/2036]
loss: 0.80646    [ 512/2036]
loss: 0.

## Now load the saved model, and check its performance

In [18]:
loaded_model = PreTrainedAllLayers(
    250,
    # dim1,
    norm_means=w2v2_mean,
    norm_std_devs=w2v2_std_dev,
)

In [19]:
loaded_dict = torch.load(
    "./saved_models/dense/best_pt_all_spkr.pt",
    map_location=device
)

In [20]:
loaded_dict["epochs_done"]

10

In [21]:
loaded_model.load_state_dict(loaded_dict["model_state_dict"])

<All keys matched successfully>

In [22]:
# On train set
test_epoch(train_dataloader, num_classes, loaded_model, loss_fn);

Test Error:
    loss: 0.19027, acc: 93.57 %
    avg recall: 93.94 %


In [23]:
# On validation set
test_epoch(val_dataloader, num_classes, loaded_model, loss_fn);

Test Error:
    loss: 0.40250, acc: 86.54 %
    avg recall: 87.65 %


In [24]:
# On test set
test_epoch(test_dataloader, num_classes, loaded_model, loss_fn);

Test Error:
    loss: 0.55501, acc: 81.73 %
    avg recall: 83.48 %
